# Install

In [ ]:
# pip install patchify

In [ ]:
# pip install Keras-Applications

In [ ]:
# pip install segmentation-models-3D

In [ ]:
# pip install scikit-image

In [ ]:
import tensorflow as tf
import keras 
print(tf.__version__)
print(keras.__version__)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
#Make sure the GPU is available. 
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
tf.test.gpu_device_name()

# STEP 1: Data Preparation
- Load dataset and slice padding
- patchify

In [ ]:
import numpy as np
from skimage import io
import os

import random
from scipy import ndimage

## Set cases

In [ ]:
# Full Train 32 cases: train 24, val 8
train_cases = ['Normal_03.tif','Normal_04.tif','Normal_05.tif','Normal_06.tif','Normal_07.tif','Normal_08.tif',
               'Mild_03.tif','Mild_04.tif','Mild_05.tif','Mild_06.tif','Mild_07.tif','Mild_08.tif',
               'Moderate_03.tif','Moderate_04.tif','Moderate_05.tif','Moderate_06.tif','Moderate_07.tif','Moderate_08.tif',
               'Severe_02.tif','Severe_03.tif','Severe_04.tif','Severe_05.tif','Severe_06.tif','Severe_08.tif']

# aug_cases = ['Normal_03.tif','Normal_07.tif',
#              'Mild_03.tif','Mild_04.tif',
#              'Moderate_03.tif','Moderate_04.tif',
#              'Severe_02.tif','Severe_04.tif']

#aug แบบ2 : แย่กว่าแบบ 1
# aug_cases = ['Normal_03.tif','Normal_07.tif',
#              'Mild_03.tif','Mild_08.tif',
#              'Moderate_03.tif','Moderate_04.tif',
#              'Severe_02.tif','Severe_04.tif']

#aug แบบ3
# aug_cases = ['Normal_07.tif',
#              'Mild_08.tif',
#              'Moderate_04.tif',
#              'Severe_04.tif']

val_cases = ['Normal_01.tif','Normal_02.tif',
             'Mild_01.tif','Mild_02.tif',
             'Moderate_01.tif','Moderate_02.tif',
             'Severe_01.tif','Severe_07.tif']

print('Train: ',len(train_cases))
# print('Augment: ',len(aug_cases))
print('Val:   ',len(val_cases))
print('Total: ',(len(train_cases)+len(val_cases)))
print(len(val_cases)/(len(train_cases)+len(val_cases)))

In [ ]:
#Padding function
def padding(image, slice_vol):
    slice_, row, col= image.shape #(0,1,2) ตรงกับ (119,256,256)
  
    #เพิ่ม slice เข้าไป
    if slice_vol > slice_: 
        slice_padding = slice_vol - slice_
        padding_array = np.zeros((slice_padding, row, col))
        image_paded = np.concatenate((image,padding_array), axis=0)
    #เ  อา slice ออก
    else: 
        image_paded = image[:slice_vol,:,:]
    return np.asarray(np.array(image_paded), dtype="uint8" )

#Load data for loop function
def load_data(list_case, drive_path,img_path,mask_path):
    image = np.array([])
    mask = np.array([])
 
    for case in list_case:
        img = io.imread(drive_path+img_path+'/'+case)
        mask_ = io.imread(drive_path+mask_path+'/'+case)
        print('image: '+case+' slice: ',img.shape[0])
        print('mask:'+case+' slice: ',mask_.shape[0])
        if image.shape[0] == 0:
            if img.shape[0] <= 128:
                image = padding(img, 128)
                mask = padding(mask_, 128)
            elif ((img.shape[0] > 128) and (img.shape[0] <= 175)):
                # set center range 
                start_lung = int((img.shape[0]/2)-64)
                end_lung = int((img.shape[0]/2)+64)
                image = np.asarray(np.array(img[start_lung:end_lung,:,:]), dtype="uint8" )
                mask = np.asarray(np.array(mask_[start_lung:end_lung,:,:]), dtype="uint8" )
            else:
                if img.shape[0] <= 256:
                    # skipping slice
                    image = padding(img[::2,:,:], 128)
                    mask = padding(mask_[::2,:,:], 128)
                else:
                    print('CT slice volume is to large')
          
        else:
            if img.shape[0] <= 128:
                image = np.concatenate((image, padding(img, 128)), axis=0)
                mask = np.concatenate((mask, padding(mask_, 128)), axis=0)
            elif ((img.shape[0] > 128) and (img.shape[0] <= 175)):
                # set center range 
                start_lung = int((img.shape[0]/2)-64)
                end_lung = int((img.shape[0]/2)+64)
                img_arr = np.asarray(np.array(img[start_lung:end_lung,:,:]), dtype="uint8" )
                mask_arr = np.asarray(np.array(mask_[start_lung:end_lung,:,:]), dtype="uint8" )
                image = np.concatenate((image, img_arr), axis=0)
                mask = np.concatenate((mask, mask_arr), axis=0)
            else:
                if img.shape[0] <= 256:
                    image = np.concatenate((image, padding(img[::2,:,:], 128)), axis=0)
                    mask = np.concatenate((mask, padding(mask_[::2,:,:], 128)), axis=0)
                else:
                    print('CT slice volume is to large')
   
    print(image.shape)
    print(mask.shape)
    return image, mask

#Load data for loop function and data augmentation
def rotate_img(volume, angle):
    augmented_volume = np.array([])
    for i in range(volume.shape[0]):
        if augmented_volume.shape[0] == 0:
            augmented_volume = ndimage.rotate(volume[i], angle, reshape=False) #scipy_rotate(volume[i], angle)
            augmented_volume = np.expand_dims(augmented_volume, axis=0)
            # print('shape:', augmented_volume.shape)
        else:
            img_aug = ndimage.rotate(volume[i], angle, reshape=False) #scipy_rotate(volume[i], angle)
            img_aug = np.expand_dims(img_aug, axis=0)
            augmented_volume = np.concatenate((augmented_volume, img_aug), axis=0)
            # print('shape:', augmented_volume.shape)
    return augmented_volume

def rotate_mask(mask_volume, angle):
    augmented_volume = np.array([])
    for i in range(mask_volume.shape[0]):
        if augmented_volume.shape[0] == 0:
            augmented_volume = ndimage.rotate(mask_volume[i], angle, order=0,reshape=False) #scipy_rotate(volume[i], angle)
            augmented_volume = np.expand_dims(augmented_volume, axis=0)
            # print('shape:', augmented_volume.shape)
        else:
            img_aug = ndimage.rotate(mask_volume[i], angle, order=0,reshape=False) #scipy_rotate(volume[i], angle)
            img_aug = np.expand_dims(img_aug, axis=0)
            augmented_volume = np.concatenate((augmented_volume, img_aug), axis=0)
            # print('shape:', augmented_volume.shape)
    return augmented_volume

def load_data_aug(list_case, drive_path,img_path,mask_path):
    image = np.array([])
    mask = np.array([])
    random.seed(20)
    # define some rotation angles
    angles = [-15, 15]
 
    for case in list_case:
        img = io.imread(drive_path+img_path+'/'+case)
        mask_ = io.imread(drive_path+mask_path+'/'+case)
        print('image: '+case+' slice: ',img.shape[0])
        print('mask:'+case+' slice: ',mask_.shape[0])

        # pick angles at random
        angle = random.choice(angles)
        print(case+' angle: ',angle)

        if image.shape[0] == 0:
            # augmentation
            img = rotate_img(img, angle)
            mask_ = rotate_mask(mask_, angle)

            # slice volumn adjust
            if img.shape[0] <= 128:
                image = padding(img, 128)
                mask = padding(mask_, 128)
            elif ((img.shape[0] > 128) and (img.shape[0] <= 175)):
                # set center range 
                start_lung = int((img.shape[0]/2)-64)
                end_lung = int((img.shape[0]/2)+64)
                image = np.asarray(np.array(img[start_lung:end_lung,:,:]), dtype="uint8" )
                mask = np.asarray(np.array(mask_[start_lung:end_lung,:,:]), dtype="uint8" )
            else:
                if img.shape[0] <= 256:
                    # skipping slice
                    image = padding(img[::2,:,:], 128)
                    mask = padding(mask_[::2,:,:], 128)
                else:
                    print('CT slice volume is to large')
          
        else:
            # augmentation
            img = rotate_img(img, angle)
            mask_ = rotate_mask(mask_, angle)

            # slice volumn adjust
            if img.shape[0] <= 128:
                image = np.concatenate((image, padding(img, 128)), axis=0)
                mask = np.concatenate((mask, padding(mask_, 128)), axis=0)
            elif ((img.shape[0] > 128) and (img.shape[0] <= 175)):
                # set center range 
                start_lung = int((img.shape[0]/2)-64)
                end_lung = int((img.shape[0]/2)+64)
                img_arr = np.asarray(np.array(img[start_lung:end_lung,:,:]), dtype="uint8" )
                mask_arr = np.asarray(np.array(mask_[start_lung:end_lung,:,:]), dtype="uint8" )
                image = np.concatenate((image, img_arr), axis=0)
                mask = np.concatenate((mask, mask_arr), axis=0)
            else:
                if img.shape[0] <= 256:
                    image = np.concatenate((image, padding(img[::2,:,:], 128)), axis=0)
                    mask = np.concatenate((mask, padding(mask_[::2,:,:], 128)), axis=0)
                else:
                    print('CT slice volume is to large')
   
    print(image.shape)
    print(mask.shape)
    return image, mask

def load_dataStartEndLung(list_case, drive_path,img_path,mask_path):
    image = np.array([])
    mask = np.array([])
 
    for case in list_case:
        img = io.imread(drive_path+img_path+'/'+case)
        mask_ = io.imread(drive_path+mask_path+'/'+case)
        print('image: '+case+' slice: ',img.shape[0])
        print('mask:'+case+' slice: ',mask_.shape[0])
        if image.shape[0] == 0:
            if img.shape[0] <= 128:
                image = padding(img, 128)
                mask = padding(mask_, 128)
            else:
                if img.shape[0] <= 256:
                    # skipping slice
                    image = padding(img[::2,:,:], 128)
                    mask = padding(mask_[::2,:,:], 128)
                else:
                    print('CT slice volume is to large')
          
        else:
            if img.shape[0] <= 128:
                image = np.concatenate((image, padding(img, 128)), axis=0)
                mask = np.concatenate((mask, padding(mask_, 128)), axis=0)
            else:
                if img.shape[0] <= 256:
                    image = np.concatenate((image, padding(img[::2,:,:], 128)), axis=0)
                    mask = np.concatenate((mask, padding(mask_[::2,:,:], 128)), axis=0)
                else:
                    print('CT slice volume is to large')
   
    print(image.shape)
    print(mask.shape)
    return image, mask

In [ ]:
Tiff_folder = 'Tif_256x256/'
# Tiff_folder = 'Tif_512x512/'
# Tiff_folder ='Tif_256x256_StartEnd/'
img_path_c1 = 'images'
img_path_c2 = 'images_createCLAHE'
img_path_c3 = 'images_equalizeHist'
mask_path = 'masks'

path_ = '/tf/Project/LungLobe/'
drive_path = path_+Tiff_folder
drive_path

## Train

In [ ]:
# image, mask = load_data(train_cases, drive_path,img_path_c1, mask_path)
cre, mask = load_data(train_cases, drive_path,img_path_c2, mask_path)
# equ, mask = load_data(train_cases, drive_path,img_path_c3, mask_path)

# image, mask = load_dataStartEndLung(train_cases, drive_path,img_path,mask_path)

In [ ]:
# Load data augmentation
# image_aug, mask_aug = load_data_aug(aug_cases, drive_path,img_path,mask_path)

In [ ]:
# print(np.unique(mask_aug))

## Validation

In [ ]:
# test_img, test_mask = load_data(val_cases, drive_path,img_path_c1,mask_path)
test_cre, test_mask = load_data(val_cases, drive_path,img_path_c2,mask_path)
# test_equ, test_mask = load_data(val_cases, drive_path,img_path_c3,mask_path)

# test_img, test_mask = load_dataStartEndLung(val_cases, drive_path,img_path,mask_path)

### Value check

In [ ]:
# print(np.unique(cre))

In [ ]:
# change image value to [0,1]
# image = image/255
# test_img = test_img/255

# cre = cre/255
# test_cre = test_cre/255

# equ = equ/255
# test_equ = test_equ/255


In [ ]:
# print(np.unique(test_cre))

In [ ]:
# print(np.unique(mask))

### Slice visualization

slice plot function

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    # data = np.rot90(np.array(data))
    # data = np.transpose(data)
    data = np.reshape(np.array(data), (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 20.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()

In [ ]:
#Visualize raw images
# plot_slices(10, 15, 256, 256, test_cre[:150, :, :]) #image, cre

In [ ]:
# mask of the CT scan.
# plot_slices(20, 20, 256, 256, mask[:400, :, :])

### Patchify

In [ ]:
from patchify import patchify, unpatchify

In [ ]:
# print('Train image:',image.shape)
print('Train image:',cre.shape)
# print('Train image:',equ.shape)
print('Train mask:',mask.shape)
# print('Augmented image:',image_aug.shape)
# print('Augmented mask:',mask_aug.shape)

# image = np.concatenate((image, image_aug), axis=0)
# mask = np.concatenate((mask, mask_aug), axis=0)
# print(image.shape)
# print(mask.shape)

In [ ]:
#z,x,y for model (128, 256, 256) Densenet121, resnet18
slice_volumn = 128 # set จำนวนสไลด์ที่จะใช้เทรน
'''
Train
'''
# img_patches = patchify(image, (slice_volumn, 256, 256), step=slice_volumn)
img_patches = patchify(cre, (slice_volumn, 256, 256), step=slice_volumn)
# img_patches = patchify(equ, (slice_volumn, 256, 256), step=slice_volumn)
mask_patches = patchify(mask, (slice_volumn, 256, 256), step=slice_volumn)

'''
Test
'''
# test_img_patches = patchify(test_img, (slice_volumn, 256, 256), step=slice_volumn)
test_img_patches = patchify(test_cre, (slice_volumn, 256, 256), step=slice_volumn)
# test_img_patches = patchify(test_equ, (slice_volumn, 256, 256), step=slice_volumn)
test_mask_patches = patchify(test_mask, (slice_volumn, 256, 256), step=slice_volumn)

print(img_patches.shape)
print(mask_patches.shape)
print(test_img_patches.shape)
print(test_mask_patches.shape)

In [ ]:
input_img = np.reshape(img_patches, (-1, img_patches.shape[3], img_patches.shape[4], img_patches.shape[5]))
input_mask = np.reshape(mask_patches, (-1, mask_patches.shape[3], mask_patches.shape[4], mask_patches.shape[5]))

input_test_img = np.reshape(test_img_patches, (-1, test_img_patches.shape[3], test_img_patches.shape[4], test_img_patches.shape[5]))
input_test_mask = np.reshape(test_mask_patches, (-1, test_mask_patches.shape[3], test_mask_patches.shape[4], test_mask_patches.shape[5]))

print(input_img.shape)  # n_patches, x, y, z
print(input_test_img.shape)

In [ ]:
# Channels
# train_img = np.stack((input_img,input_img_CLAHE,input_img_equalizeHist), axis=-1)
train_img = np.stack((input_img,)*3, axis=-1)
train_mask = np.expand_dims(input_mask, axis=4)

# img_test = np.stack((input_test_img,input_test_img_CLAHE,input_test_img_equalizeHist), axis=-1)
img_test = np.stack((input_test_img,)*3, axis=-1)
mask_test = np.expand_dims(input_test_mask, axis=4)

print(train_img.shape)
print(train_mask.shape)
print(img_test.shape)
print(mask_test.shape)

to_categorical

In [ ]:
from tensorflow.keras.utils import to_categorical

n_classes = 6

y_train = to_categorical(train_mask, num_classes=n_classes)
y_test = to_categorical(mask_test, num_classes=n_classes)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train = train_img
X_test = img_test
# y_train = train_mask_cat
# y_test = test_mask_cat

# raw image
print(X_train.shape)
print(X_test.shape)

# mask
print(y_train.shape)
print(y_test.shape)

# STEP 2: Model

### 3Dunet-Densenet121, 3DFPN

In [ ]:
from tensorflow import keras
import segmentation_models_3D as sm
# from segmentation_models.losses import CategoricalCELoss

encoder_weights = 'imagenet'

#Try densenet121, densenet169, densenet201 Total params: 22ล้าน
# resnet18, resnet34, resnet50, resnet101, resnet152
#inceptionv3, efficientnetb0
BACKBONE = 'resnet152'
activation = 'softmax'
patch_size = 256 # 128, 224, 256, 512
volumn_size = slice_volumn # 32, 64, 128
n_classes = 6
channels=3

LR = 0.0001
optim = keras.optimizers.Adam(LR)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
preprocess_input = sm.get_preprocessing(BACKBONE)

preprocess_input

In [ ]:
X_train_prep = preprocess_input(X_train)
X_test_prep = preprocess_input(X_test)

Build model

In [ ]:
model = sm.Unet(BACKBONE, classes=n_classes, 
                input_shape=(volumn_size, patch_size, patch_size, channels), 
                encoder_weights=encoder_weights,
                #encoder_freeze = True, # set เพื่อทำการ Tuning
                activation=activation,
                dropout=0.2)
# model = sm.FPN(BACKBONE, classes=n_classes, 
#                input_shape=(volumn_size, patch_size, patch_size, channels), 
#                encoder_weights=encoder_weights,
#                activation=activation,
#                pyramid_dropout=0.2)

Add regularizer

In [ ]:
import tensorflow as tf
from segmentation_models_3D.utils import set_regularization
'''
    set_regularization(model,
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    beta_regularizer=None,
    gamma_regularizer=None,
    **kwargs)
'''

# l2=0.01 ioU=0.83
# l2=0.1
regularizer = tf.keras.regularizers.L2(l2=0.01)
# regularizer = tf.keras.regularizers.L1(l1=0.01)
set_regularization(model, kernel_regularizer = regularizer,
                   bias_regularizer = regularizer)
# set_regularization(model, activity_regularizer = regularizer) #ใช้ไม่ได้นะเวลา train ละจะ error

compile

In [ ]:
# aliases
# jaccard_loss = JaccardLoss()
# dice_loss = DiceLoss()

# binary_focal_loss = BinaryFocalLoss()
# categorical_focal_loss = CategoricalFocalLoss()
# binary_crossentropy = BinaryCELoss()
# categorical_crossentropy = CategoricalCELoss()

# loss combinations
# categorical_focal_dice_loss = categorical_focal_loss + dice_loss ตอนนี้ใช้อันนี้อยู่
# categorical_focal_jaccard_loss = categorical_focal_loss + jaccard_loss
# cce_dice_loss = categorical_crossentropy + dice_loss
# cce_jaccard_loss = categorical_crossentropy + jaccard_loss

In [ ]:
# dice_loss = DiceLoss()
# categorical_focal_loss = CategoricalFocalLoss()
# categorical_crossentropy = CategoricalCELoss()
# categorical_focal_dice_loss = categorical_focal_loss + dice_loss ตอนนี้ใช้อันนี้อยู่
# cce_dice_loss = categorical_crossentropy + dice_loss

total_loss = sm.losses.categorical_focal_dice_loss
model.compile(optimizer = optim, loss=total_loss, metrics=metrics)
print(model.summary())

Visualize Model Struture

In [ ]:
# !pip install pydot
# !pip install graphviz

In [ ]:
# from keras.utils.vis_utils import plot_model
# path_ = '/tf/Project/LungLobe/'
# Model_name = '3DUNet_densenet121_Model1'
# save_path_model_struture = path_+'LungModel/'+Model_name+'.png'

# plot_model(model, to_file=save_path_model_struture, show_shapes=True, show_layer_names=True)

Fit model Densenet121-3DUnet

In [ ]:
# Densenet121-3DUnet, Densenet169-3DUnet #เทรนทุกสไลด์ Tiff_256x256
# class:6 : 1 background+ 5 lobe
# เทรน 200 Epochs โดยมี EarlyStopping
import time
import datetime
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10) #20

path_ = '/tf/Project/LungLobe/'
Model_name = '3DUNet_resnet152_Model46'
save_path_ = path_+'LungModel/'+Model_name+'.h5'
mc = ModelCheckpoint(save_path_, monitor='val_iou_score', mode='max', verbose=1, save_best_only=True)

start = time.time()
#-------------------------------------------
# train 24 val 8
history=model.fit(X_train_prep, y_train, batch_size=1, 
          epochs=100, verbose=1,
          validation_data=(X_test_prep, y_test),
          shuffle=True,
          callbacks=[es, mc])
#-------------------------------------------
end = time.time()
print('time process used>', str(datetime.timedelta(seconds=end - start)))

save history

In [ ]:
import pickle

save_model_p = path_+'LungModel/'+'History_'+Model_name
with open(save_model_p, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)